# Задание
Ваша задача — реализовать класс Account, который моделирует поведение
банковского счёта. Этот класс должен не только выполнять базовые операции,
но и вести детальный учёт всех действий, а также предоставлять аналитику по
истории операций.


## Этап 1. Реализация базового класса Account ##
Класс должен быть инициализирован с параметрами:
- account_holder (str): имя владельца счёта;
- balance (float, по умолчанию 0): начальный баланс счёта, не может быть отрицательным.
### Атрибуты:
- holder: хранит имя владельца;
- _balance: приватный атрибут для хранения текущего баланса;
operations_history: список или другая структура для хранения истории
операций.
### Важно: #
Каждая операция должна храниться не просто как число, а как
структурированная информация, например, словарь или кортеж. Минимальный набор
данных для операции: тип операции ('deposit' или 'withdraw'), сумма, дата и время
операции, текущий баланс после операции, статус ('success' или 'fail')

In [ ]:
from datetime import datetime
from typing import List, Dict, Any, Union

In [3]:
class Account:


    def __init__(self, account_holder: str, balance: float = 0.0):
        if balance < 0:
            raise ValueError("Начальный баланс не может быть отрицательным.")

        self.holder: str = account_holder
        self._balance: float = balance
        # История операций: список словарей
        # Каждая запись: {'type', 'amount', 'timestamp', 'new_balance', 'status'}
        self.operations_history: List[Dict[str, Any]] = []

        # Добавляем запись об открытии счёта в историю
        self._record_operation(
            op_type='initial_deposit',
            amount=balance,
            status='success',
            new_balance=balance
        )

## Этап 2. Реалиазация методов ##
1. __init__(self, account_holder, balance=0): конструктор;
2. deposit(self, amount): метод для пополнения счёта:
- принимает сумму (должна быть положительной);
- в случае успеха обновляет баланс и добавляет запись в историю операций.
3. withdraw(self, amount): метод для снятия средств:
- принимает сумму (должна быть положительной);
- проверяет, достаточно ли средств на счёте, если нет — операция не
проходит, но ее попытка с статусом 'fail' все равно фиксируется в истории;
- в случае успеха обновляет баланс и добавляет запись.
4. get_balance(self): метод, который возвращает текущий баланс.
5. get_history(self): метод, который возвращает историю операций.
### Важно: # 
продумайте, в каком формате его вернуть. Для работы с датой и временем
используйте модуль datetime. Получить текущее время можно с помощью
datetime.now().

In [4]:
class Account:
    # ... (Атрибуты: holder, _balance, operations_history)

    # 1. __init__(self, account_holder, balance=0): конструктор
    def __init__(self, account_holder: str, balance: float = 0.0):
        if balance < 0:
            raise ValueError("Начальный баланс не может быть отрицательным.")

        self.holder = account_holder
        self._balance = balance
        self.operations_history = []
        # Вспомогательный метод для записи истории
        # self._record_operation(...) 

    # 2. deposit(self, amount): метод для пополнения счёта
    def deposit(self, amount: float):
        if amount <= 0:
            self._record_operation('deposit', amount, 'fail', message="Сумма пополнения должна быть положительной.")
            return False

        self._balance += amount
        self._record_operation('deposit', amount, 'success')
        return True

    # 3. withdraw(self, amount): метод для снятия средств
    def withdraw(self, amount: float):
        if amount <= 0:
            self._record_operation('withdraw', amount, 'fail', message="Сумма снятия должна быть положительной.")
            return False

        if self._balance >= amount:
            self._balance -= amount
            self._record_operation('withdraw', amount, 'success')
            return True
        else:
            self._record_operation('withdraw', amount, 'fail', message="Недостаточно средств на счёте.")
            return False

    # 4. get_balance(self): метод, который возвращает текущий баланс
    def get_balance(self) -> float:
        return self._balance

    # 5. get_history(self): метод, который возвращает историю операций
    def get_history(self) -> list:
        return self.operations_history

    # Примечание: Вспомогательный метод для истории
    def _record_operation(self, op_type: str, amount: float, status: str, message: str = None, **kwargs):
        record = {
            'type': op_type,
            'amount': amount,
            'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'status': status,
            'current_balance': self._balance,
        }
        if message:
            record['message'] = message
        record.update(kwargs)
        self.operations_history.append(record)


## Этап 3. Реализация наследования. ##
Создайте класс CreditAccount(Account), который наследует всю функциональность
класса Account и добавляет новую.
### Особенности кредитного счёта:
1. При инициализации принимает дополнительный параметр credit_limit
(кредитный лимит). Баланс такого счета может быть отрицательным, но не ниже
значения -credit_limit.
2. По запросу показывает, сколько кредитных средств еще доступно (текущий
баланс + кредитный лимит).
3. В историю операций добавляется информацию о том, были ли использованы
кредитные средства в данной операции.

In [6]:
class CreditAccount(Account):

    # 1. Инициализация с credit_limit и проверка баланса
    def __init__(self, account_holder: str, credit_limit: float, balance: float = 0.0):
        if credit_limit <= 0:
            raise ValueError("Кредитный лимит должен быть положительным.")
        
        # Проверка, что начальный баланс не ниже максимально допустимого долга
        if balance < -credit_limit:
            raise ValueError(f"Начальный баланс не может быть ниже -{credit_limit:.2f}")

        # Вызываем конструктор родителя
        super().__init__(account_holder, balance)
        self.credit_limit = credit_limit
        print(f"Кредитный лимит установлен: {self.credit_limit:.2f}")

    # 2. Метод для показа доступных кредитных средств
    def get_available_credit(self) -> float:
        # Если баланс положительный, доступный кредит равен лимиту.
        # Если баланс отрицательный (долг), доступный кредит = лимит - |долг|.
        return self.credit_limit + self._balance
            
    # 3. Учет использования кредитных средств
    def withdraw(self, amount: float):
        # ... (Проверки на amount > 0 и лимит)

        min_allowed_balance = -self.credit_limit
        new_balance = self._balance - amount
        
        if new_balance >= min_allowed_balance:
            
            # --- Логика определения использования кредита ---
            used_credit = False
            # Если текущего баланса не хватает для покрытия снятия,
            # или если баланс уже был отрицательным, кредит использован.
            if self._balance < amount:
                used_credit = True
            
            self._balance = new_balance
            
            # --- Добавление информации в историю ---
            self._record_operation(
                'withdraw', 
                amount, 
                'success', 
                used_credit=used_credit,  # <-- Вот этот параметр
                credit_limit=self.credit_limit
            )
            return True
